In [1]:
from prevo.viewers import TkWindow, TkViewer
from prevo.viewers import CvWindow, CvViewer
from prevo.misc import PeriodicSensor
from threading import Thread, Event
import oclock
import numpy as np
%matplotlib tk

# Dummy camera sensor

The section below is just to define a dummy class that mimicks a camera sending images on a queue.

In [2]:
class LapseCamera(PeriodicSensor):
    """Mock time-lapse camera returning white-noise images periodically"""
    
    name = 'Mock Lapse Camera'
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num = 0
    
    def _read(self):
        """Return image in a dict (see explanation below)"""
        img = np.random.randint(256, size=(480, 640), dtype='uint8')
        data = {'image': img, 'num': self.num}
        self.num += 1
        return data

In [3]:
camera1 = LapseCamera(interval=0.04)
camera1.start()

camera2 = LapseCamera(interval=2)
camera2.start()

# Tkinter

In [4]:
win1 = TkWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = TkWindow(camera2.queue, name='Camera 2', show_num=True)

In [5]:
viewer = TkViewer(windows=(win1, win2))
viewer.start()

# OpenCV

In [6]:
# Create thread that will automatically sent a stop request to the viewer after 10 seconds

e_stop = Event()


def stop_display():
    timer = oclock.Timer()
    while timer.elapsed_time < 10:
        timer.checkpt()
    print('STOP REQUESTED')
    e_stop.set()

    
Thread(target=stop_display).start()

In [7]:
win1 = CvWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = CvWindow(camera2.queue, name='Camera 2', show_num=True)

In [8]:
viewer = CvViewer(windows=(win1, win2), external_stop=e_stop)
viewer.start()

STOP REQUESTED
